In [1]:
from lxml import etree, html
import pandas as pd
from datetime import date
import requests
import re

In [2]:
doc = etree.parse('BOE-A-2020-14372.xml')

In [3]:
#etree.tostring(doc,pretty_print=True ,xml_declaration=True, encoding="utf-8")

In [4]:
raiz=doc.getroot()
raiz.tag

'documento'

In [5]:
len(raiz)

4

In [6]:
libro=raiz[0]
libro.tag

'metadatos'

In [7]:
libro=raiz[0]
libro[0].text

'BOE-A-2020-14372'

In [8]:
libro=raiz[1]
libro[0].tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [9]:
libro=raiz[1]
libro[0].text

'\n            '

In [10]:
Identificador=raiz.find("metadatos/identificador")
Titulo=raiz.find("metadatos/titulo")
URL_pdf=raiz.find("metadatos/url_pdf")

In [11]:
print(Identificador.text)
print(Titulo.text)
print("https://www.boe.es" + URL_pdf.text)


BOE-A-2020-14372
Decreto-ley 35/2020, de 20 de octubre, de modificación del Decreto-ley 19/2020, de 19 de mayo, y del Decreto-ley 29/2020, de 28 de julio, en materia de adopción de medidas sociales y sanitarias para paliar los efectos de la pandemia generada por la COVID-19.
https://www.boe.es/boe/dias/2020/11/18/pdfs/BOE-A-2020-14372.pdf


In [12]:
materia=raiz.find("analisis/materias/materia")
materia.text

'Asistencia sanitaria'

In [13]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asistencia sanitaria
Asistencia social
Ayudas
Cataluña
Centros residenciales para personas con discapacidad
Epidemias
Establecimientos residenciales para la Tercera Edad
Pagos
Servicios Sociales de las Comunidades Autónomas


In [14]:
type(materias)

list

In [15]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Asuntos sociales
Sanidad


# Recoge resumen diario del BOE de hoy

In [16]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)



Fecha de Hoy = 20210120
dia = 20
mes = 01
año = 2021


In [17]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [18]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210120'

In [19]:
#pip install requests for installation



url = URL_XML_resumen
r = requests.get(url)
#r.content
#r.text

In [20]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
#save_html(r.content, './BOEs/resumen.xml')
save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [21]:
#r.encoding

In [22]:
#resumen = etree.parse('./BOEs/resumen.xml')
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [23]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [24]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [25]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']


In [26]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [27]:
tabla_resumen = pd.DataFrame()

In [28]:
#for seccion in raiz.xpath('//seccion[contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "I. Disposiciones generales")]'):
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
#for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
                #print(item_urlXml)

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [29]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 1/2021, de 19 de enero, de p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-793],[I. Disposiciones generales]
1,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sector lácteo],"[Real Decreto 24/2021, de 19 de enero, por el ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-794],[I. Disposiciones generales]
2,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sanidad animal],"[Orden APA/19/2021, de 18 de enero, por la que...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-795],[I. Disposiciones generales]
3,[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Aparatos eléctricos y electrónicos. Residuos],"[Real Decreto 27/2021, de 19 de enero, por el ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-796],[I. Disposiciones generales]
4,[COMUNIDAD AUTÓNOMA DE ANDALUCÍA],[Presupuestos],"[Ley 3/2020, de 28 de diciembre, del Presupues...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-797],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [30]:
for item_URL in tabla_resumen['Item_URL_XML']:
    #print(URL)
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-793']
<re.Match object; span=(0, 14), match='BOE-A-2021-793'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-794']
<re.Match object; span=(0, 14), match='BOE-A-2021-794'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-795']
<re.Match object; span=(0, 14), match='BOE-A-2021-795'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-796']
<re.Match object; span=(0, 14), match='BOE-A-2021-796'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-797']
<re.Match object; span=(0, 14), match='BOE-A-2021-797'>


In [31]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2020-15394.xml', './BOEs/BOE-A-2020-16676.xml', './BOEs/BOE-A-2021-590.xml', './BOEs/BOE-A-2020-14465.xml', './BOEs/BOE-A-2020-14459.xml', './BOEs/BOE-A-2020-15237.xml', './BOEs/BOE-A-2020-15223.xml', './BOEs/BOE-A-2020-15222.xml', './BOEs/BOE-A-2020-15236.xml', './BOEs/BOE-A-2020-14458.xml', './BOEs/BOE-A-2020-14464.xml', './BOEs/BOE-A-2021-591.xml', './BOEs/BOE-A-2020-15395.xml', './BOEs/BOE-A-2021-793.xml', './BOEs/BOE-A-2020-15397.xml', './BOEs/BOE-A-2020-16675.xml', './BOEs/BOE-A-2021-587.xml', './BOEs/BOE-A-2020-14466.xml', './BOEs/BOE-A-2020-15208.xml', './BOEs/BOE-A-2020-14880.xml', './BOEs/BOE-A-2020-15220.xml', './BOEs/BOE-A-2020-15234.xml', './BOEs/BOE-A-2020-15235.xml', './BOEs/BOE-A-2020-15221.xml', './BOEs/BOE-A-2020-15209.xml', './BOEs/BOE-A-2020-14467.xml', './BOEs/BOE-A-2021-592.xml', './BOEs/BOE-A-2020-16674.xml', './BOEs/BOE-A-2020-15396.xml', './BOEs/BOE-A-2020-15392.xml', './BOEs/BOE-A-2021-794.xml', './BOEs/BOE-A-2020-16843.xml', './BOEs/BOE-A-2021-

In [32]:
#import os
#path = './BOEs'
#BOEs = [f for f in os.listdir(path) if f.startswith('BOE')]
#BOEs

In [33]:
tabla_materias = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]
    
    #Item_Id = '[' + BOE[7:23] + ']'
    #Item_Id = raiz.find('metadatos/identificador')
    Item_Id = raiz.xpath(".//identificador/text()")
 
    #print(Item_Id, materias, alertas)
    #tabla_materias.loc[['Item_id'], ['Materias']] = materias
    
    #tabla_materias = tabla_materias.append({'Item_id': Item_Id,
    #                                        'Materias' : materias,
    #                                        'Alertas' : alertas},
    #                                        ignore_index=True)

    tabla_materias = tabla_materias.append({'Item_id': Item_Id,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


    #tabla_materias.Item_id.apply(list)
    #fila_tabla = tabla_resumen.set_index('Item_id').filter(like=BOE[18:23], axis=0).reset_index()
    #print(fila_tabla['Item_id'])

    #tabla_resumen.loc[fila_tabla['Item_id']]['Materias'] = materias


In [34]:
tabla_materias.sort_values('Item_id')

,Alertas,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto
207,"[Relaciones internacionales, Sanidad, Sistema ...",[BOE-A-2020-14457],"[Acuerdos internacionales, Créditos Presupuest...",[BOE-A-2002-15339],[DE CONFORMIDAD con],[el Acuerdo de 12 de septiembre de 2001]
9,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14458],"[Acuerdos internacionales, Barcelona, Centros ...",[BOE-A-2020-13842],[CORRECCIÓN de errores],"[, con rectificación del título, de las Notas ..."
4,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14459],"[Acuerdos internacionales, Barcelona, Centros ...",[BOE-A-2020-13843],[CORRECCIÓN de errores],"[, con rectificación del título, de las Notas ..."
51,"[Administración de Justicia, Sistema financiero]",[BOE-A-2020-14460],"[Ayuntamientos, Créditos Presupuestarios, Juzg...","[BOE-A-2018-9268, BOE-A-1992-28826, BOE-A-1988...","[DE CONFORMIDAD con, CITA, CITA]","[la Ley 6/2018, de 3 de julio, Ley 39/1992, de..."
58,[Sistema financiero],[BOE-A-2020-14461],[Administración y Contabilidad de la Hacienda ...,[BOE-A-2003-21614],[DE CONFORMIDAD con],"[la Ley 47/2003, de 26 de noviembre]"
...,...,...,...,...,...,...
13,"[Asuntos sociales, Comercio, Consumidores y us...",[BOE-A-2021-793],"[Aguas residuales, Autoridades Portuarias, Com...","[BOE-A-2020-17339, BOE-A-2001-14276]","[DEROGA, MODIFICA]",[la disposición adicional 6 y MODIFICA el anex...
30,"[Alimentación, Comercio, Derecho Administrativ...",[BOE-A-2021-794],"[Agencias estatales, Comercialización, Etiquet...",[],[],[]
45,"[Ganadería y animales, Medio ambiente]",[BOE-A-2021-795],"[Aves, Aves de corral, Comunidades Autónomas, ...",[],[],[]
62,"[Comercio, Consumidores y usuarios, Industria,...",[BOE-A-2021-796],"[Acumulador eléctrico, Aparatos de uso domésti...","[BOE-A-2020-7438, BOE-A-2018-9466, BOE-A-2008-...","[MODIFICA, MODIFICA, MODIFICA, MODIFICA, TRANS...","[el art. 17.3 del Real Decreto 646/2020, de 7 ..."


In [35]:
print(raiz.tag)

documento


In [36]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

[]


In [37]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asociaciones
Fundaciones
Medio ambiente
Ministerio para la Transición Ecológica y el Reto Demográfico
Subvenciones


In [38]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Medio ambiente


In [39]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

Empty DataFrame
Columns: [Departamento, Epigrafe, Item_Nombre, Item_URL_XML, Item_id, Seccion]
Index: []


In [40]:
BOE[7:23]

'BOE-A-2020-15399'

In [41]:
tabla_resumen['Item_id']

0    [BOE-A-2021-793]
1    [BOE-A-2021-794]
2    [BOE-A-2021-795]
3    [BOE-A-2021-796]
4    [BOE-A-2021-797]
Name: Item_id, dtype: object

In [42]:
tabla_materias['Item_id']

0      [BOE-A-2020-15394]
1      [BOE-A-2020-16676]
2        [BOE-A-2021-590]
3      [BOE-A-2020-14465]
4      [BOE-A-2020-14459]
              ...        
214    [BOE-A-2020-15206]
215    [BOE-A-2020-15212]
216      [BOE-A-2021-589]
217    [BOE-A-2020-15400]
218    [BOE-A-2020-15399]
Name: Item_id, Length: 219, dtype: object

In [43]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [44]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion
Item_id,,,,,


In [45]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [46]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [47]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 1/2021, de 19 de enero, de p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-793],[I. Disposiciones generales]
1,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sector lácteo],"[Real Decreto 24/2021, de 19 de enero, por el ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-794],[I. Disposiciones generales]
2,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sanidad animal],"[Orden APA/19/2021, de 18 de enero, por la que...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-795],[I. Disposiciones generales]
3,[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Aparatos eléctricos y electrónicos. Residuos],"[Real Decreto 27/2021, de 19 de enero, por el ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-796],[I. Disposiciones generales]
4,[COMUNIDAD AUTÓNOMA DE ANDALUCÍA],[Presupuestos],"[Ley 3/2020, de 28 de diciembre, del Presupues...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-797],[I. Disposiciones generales]


In [48]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [49]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_materias['Item_id'] = tabla_materias['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_materias.set_index('Item_id'))


In [50]:
tabla_resultados

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion,Alertas,Materias,Referencias,Referencias_palabra,Referencias_texto
Item_id,,,,,,,,,,
['BOE-A-2021-793'],[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 1/2021, de 19 de enero, de p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Asuntos sociales, Comercio, Consumidores y us...","[Aguas residuales, Autoridades Portuarias, Com...","[BOE-A-2020-17339, BOE-A-2001-14276]","[DEROGA, MODIFICA]",[la disposición adicional 6 y MODIFICA el anex...
['BOE-A-2021-794'],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sector lácteo],"[Real Decreto 24/2021, de 19 de enero, por el ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Alimentación, Comercio, Derecho Administrativ...","[Agencias estatales, Comercialización, Etiquet...",[],[],[]
['BOE-A-2021-795'],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sanidad animal],"[Orden APA/19/2021, de 18 de enero, por la que...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Ganadería y animales, Medio ambiente]","[Aves, Aves de corral, Comunidades Autónomas, ...",[],[],[]
['BOE-A-2021-796'],[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Aparatos eléctricos y electrónicos. Residuos],"[Real Decreto 27/2021, de 19 de enero, por el ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Comercio, Consumidores y usuarios, Industria,...","[Acumulador eléctrico, Aparatos de uso domésti...","[BOE-A-2020-7438, BOE-A-2018-9466, BOE-A-2008-...","[MODIFICA, MODIFICA, MODIFICA, MODIFICA, TRANS...","[el art. 17.3 del Real Decreto 646/2020, de 7 ..."
['BOE-A-2021-797'],[COMUNIDAD AUTÓNOMA DE ANDALUCÍA],[Presupuestos],"[Ley 3/2020, de 28 de diciembre, del Presupues...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Cultura y ocio, Derecho Administrativo, Sanid...","[Acción concertada, Administración Local, Anda...","[BOE-A-2018-1098, BOE-A-2016-957, BOE-A-2016-3...","[DEROGA, DEROGA, DEROGA, DEROGA, MODIFICA, MOD...","[la disposición adicional 11 de la Ley 5/2017,..."


In [51]:
#tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)
tabla_resultados.to_csv('./BOEs/Resultados-BOE-' + hoy + '.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [52]:
# tabla_resultados.Alertas.apply(pd.Series)\
#     .merge(tabla_resultados, left_index = True, right_index = True) \
#         .drop(["Alertas"], axis = 1) \
#             .reset_index() \
#                 .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias','Referencias','Referencias_palabra','Referencias_texto'], value_name = "Alerta") \
#                     .drop("variable", axis = 1) \
#                     .dropna() \
#                     .sort_values('Alerta')
                            

In [53]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias','Referencias','Referencias_palabra','Referencias_texto'], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

,Item_id,Item_Nombre,Seccion,Departamento,Epigrafe,Item_URL_XML,Materias,Referencias,Referencias_palabra,Referencias_texto,Alerta
1,['BOE-A-2021-794'],"[Real Decreto 24/2021, de 19 de enero, por el ...",[I. Disposiciones generales],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sector lácteo],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agencias estatales, Comercialización, Etiquet...",[],[],[],Alimentación
0,['BOE-A-2021-793'],"[Real Decreto-ley 1/2021, de 19 de enero, de p...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aguas residuales, Autoridades Portuarias, Com...","[BOE-A-2020-17339, BOE-A-2001-14276]","[DEROGA, MODIFICA]",[la disposición adicional 6 y MODIFICA el anex...,Asuntos sociales
3,['BOE-A-2021-796'],"[Real Decreto 27/2021, de 19 de enero, por el ...",[I. Disposiciones generales],[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Aparatos eléctricos y electrónicos. Residuos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acumulador eléctrico, Aparatos de uso domésti...","[BOE-A-2020-7438, BOE-A-2018-9466, BOE-A-2008-...","[MODIFICA, MODIFICA, MODIFICA, MODIFICA, TRANS...","[el art. 17.3 del Real Decreto 646/2020, de 7 ...",Comercio
5,['BOE-A-2021-793'],"[Real Decreto-ley 1/2021, de 19 de enero, de p...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aguas residuales, Autoridades Portuarias, Com...","[BOE-A-2020-17339, BOE-A-2001-14276]","[DEROGA, MODIFICA]",[la disposición adicional 6 y MODIFICA el anex...,Comercio
6,['BOE-A-2021-794'],"[Real Decreto 24/2021, de 19 de enero, por el ...",[I. Disposiciones generales],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sector lácteo],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agencias estatales, Comercialización, Etiquet...",[],[],[],Comercio
10,['BOE-A-2021-793'],"[Real Decreto-ley 1/2021, de 19 de enero, de p...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aguas residuales, Autoridades Portuarias, Com...","[BOE-A-2020-17339, BOE-A-2001-14276]","[DEROGA, MODIFICA]",[la disposición adicional 6 y MODIFICA el anex...,Consumidores y usuarios
8,['BOE-A-2021-796'],"[Real Decreto 27/2021, de 19 de enero, por el ...",[I. Disposiciones generales],[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Aparatos eléctricos y electrónicos. Residuos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acumulador eléctrico, Aparatos de uso domésti...","[BOE-A-2020-7438, BOE-A-2018-9466, BOE-A-2008-...","[MODIFICA, MODIFICA, MODIFICA, MODIFICA, TRANS...","[el art. 17.3 del Real Decreto 646/2020, de 7 ...",Consumidores y usuarios
4,['BOE-A-2021-797'],"[Ley 3/2020, de 28 de diciembre, del Presupues...",[I. Disposiciones generales],[COMUNIDAD AUTÓNOMA DE ANDALUCÍA],[Presupuestos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acción concertada, Administración Local, Anda...","[BOE-A-2018-1098, BOE-A-2016-957, BOE-A-2016-3...","[DEROGA, DEROGA, DEROGA, DEROGA, MODIFICA, MOD...","[la disposición adicional 11 de la Ley 5/2017,...",Cultura y ocio
9,['BOE-A-2021-797'],"[Ley 3/2020, de 28 de diciembre, del Presupues...",[I. Disposiciones generales],[COMUNIDAD AUTÓNOMA DE ANDALUCÍA],[Presupuestos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acción concertada, Administración Local, Anda...","[BOE-A-2018-1098, BOE-A-2016-957, BOE-A-2016-3...","[DEROGA, DEROGA, DEROGA, DEROGA, MODIFICA, MOD...","[la disposición adicional 11 de la Ley 5/2017,...",Derecho Administrativo
11,['BOE-A-2021-794'],"[Real Decreto 24/2021, de 19 de enero, por el ...",[I. Disposiciones generales],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Sector lácteo],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agencias estatales, Comercialización, Etiquet...",[],[],[],Derecho Administrativo


In [54]:
tabla_resultados['Epigrafe'].str.join(',')

Item_id
['BOE-A-2021-793']                                Medidas urgentes
['BOE-A-2021-794']                                   Sector lácteo
['BOE-A-2021-795']                                  Sanidad animal
['BOE-A-2021-796']    Aparatos eléctricos y electrónicos. Residuos
['BOE-A-2021-797']                                    Presupuestos
Name: Epigrafe, dtype: object

In [55]:
from pprint import pprint

## Convierte estructura XML en diccionario

In [56]:
BOE_XML = etree.parse('./BOEs/BOE-A-2021-756.xml')
raiz=BOE_XML.getroot()

In [57]:
result_list = [item.text for item in raiz.findall("./analisis/referencias/anteriores/anterior/palabra")]
#result_list = [{k: item.get(k) for k in item.iter('materia')} for item in raiz.findall("./analisis/materias")]
#result_list = [{{i.get('k') for i in item.iter('materia')}
#                for k in ('materia')}
#               for item in tree.findall("./analisis/materias")]
#result_list = raiz.findall("./analisis/materias")
pprint(result_list)

['DE CONFORMIDAD con', 'CITA']


In [98]:
lst = raiz.findall("./analisis/referencias/anteriores/anterior")
d = [{x.tag: {z.tag : z.text for z in x.iter()} for x in c} for c in lst]
pprint(d)

[{'palabra': {'palabra': 'DE CONFORMIDAD con'},
  'texto': {'texto': 'el art. 139.1 del Estatuto aprobado por Ley Orgánica '
                     '6/2006, de 19 de julio'}},
 {'palabra': {'palabra': 'CITA'},
  'texto': {'texto': 'Ley 12/2008, de 31 de julio'}}]


In [99]:
lst = raiz.findall("./analisis/referencias/anteriores/anterior")
d = [{x.tag: x.text for x in c.iter()} for c in lst]
pprint(d)

[{'anterior': '\n          ',
  'palabra': 'DE CONFORMIDAD con',
  'texto': 'el art. 139.1 del Estatuto aprobado por Ley Orgánica 6/2006, de 19 '
           'de julio'},
 {'anterior': '\n          ',
  'palabra': 'CITA',
  'texto': 'Ley 12/2008, de 31 de julio'}]


In [58]:
d = [{x.tag: x.text for x in c} for c in raiz]
pprint(d)

[{'departamento': 'Comunidad Autónoma de Cataluña',
  'diario': 'Boletín Oficial del Estado',
  'diario_numero': '16',
  'estado_consolidacion': None,
  'estatus_derogacion': 'S',
  'estatus_legislativo': 'L',
  'fecha_derogacion': '20201218',
  'fecha_disposicion': '20201117',
  'fecha_publicacion': '20210119',
  'fecha_vigencia': '20201120',
  'identificador': 'BOE-A-2021-756',
  'judicialmente_anulada': 'N',
  'letra_imagen': 'A',
  'numero_oficial': '45/2020',
  'origen_legislativo': 'Autonómico',
  'pagina_final': '4516',
  'pagina_inicial': '4511',
  'rango': 'Decreto-ley',
  'seccion': '1',
  'subseccion': None,
  'suplemento_letra_imagen': None,
  'suplemento_pagina_final': None,
  'suplemento_pagina_inicial': None,
  'titulo': 'Decreto-ley 45/2020, de 17 de noviembre, sobre la habilitación '
            'transitoria y extraordinaria para continuar la prestación del '
            'servicio de inspección técnica de vehículos.',
  'url_eli': 'https://www.boe.es/eli/es-ct/dl/2020/

In [59]:
pprint(raiz[2].tag)

'analisis'


In [60]:
parrafos=raiz.findall("texto/p")
for parrafo in parrafos:
    print(parrafo.text)

El artículo 67.6.a) del Estatuto prevé que los decretos ley sean promulgados, en nombre del Rey, por el Presidente de la Generalidad.
De conformidad con lo dispuesto en el artículo 6 y concordantes de la Ley 13/2008, de 5 de noviembre, de la presidencia de la Generalidad y del Gobierno, y dado el Decreto 114/2020, de 30 de septiembre, de sustitución del presidente de la Generalidad de Cataluña;
De acuerdo con ello, promulgo el siguiente Decreto-ley.
EXPOSICIÓN DE MOTIVOS
La inspección técnica de vehículos (ITV) tiene por objeto principal la comprobación periódica de que los vehículos cumplen las condiciones de seguridad y ambientales exigidas por la normativa aplicable en el momento de su homologación o, en su caso, adaptación. También tiene por objeto otras inspecciones en el supuesto de reformas y otros trámites.
La finalidad de la inspección técnica de vehículos es detectar deficiencias mecánicas que pueden comprometer la seguridad vial y, por ello, tiene carácter obligatorio, estab

# DOGC

In [61]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [62]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [63]:
r.html.links

{'/ca/index.html?newLang=es_ES&language=es_ES',
 '/ca/pdogc_serveis/cerca-republica/',
 '/ca/pdogc_serveis/pdogc_dogc_mobi/',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/pdogc_base_de_dades_del_dogc/',
 '/es/pdogc_canals_interns/pdogc_cercador_de_normativa/?destParam=cercaAdv',
 '/es/pdogc_canals_interns/pdogc_que_es_el_codi_de_verificacio/',
 '/es/pdogc_eadop/',
 '/es/pdogc_enllacos/',
 '/es/pdogc_preguntes_frequents/',
 '/es/pdogc_serveis/',
 '/es/pdogc_serveis/pdogc_normes_urbanistiques_1985-2002/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/pdogc_alertes_dogc/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/Publicacio_Anuncis_situacio_CoV-2/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/pdogc_dogc_eacat/index.html',
 '/es/pdogc_sobre_el_dogc/',
 'http://bop.diba.es/',
 'http://llibreria.gencat.cat/index.php?language=es',
 'http://web.gencat.cat/es/temes/',
 'http://www.boe

In [64]:
r.html.xpath('//*[@id="titol"]/span/text()')

[' DOGC núm.\r\n\t\t\t\t\t 8321 (20/01/2021)']

In [65]:
r.html.xpath('//*[@id="titol"]/h3/text()')


['\r\n\t\t\t\t\tÚltimo DOGC publicado\r\n\t\t\t\t']

In [66]:
r.html.lxml.tag

'html'

In [67]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[<Element span at 0x7fda5cc9dc20>]

In [68]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


[' DOGC núm.\r\n\t\t\t\t\t 8321 (20/01/2021)']


In [69]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [70]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

['anexos']

In [71]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

['1']

In [72]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[<Element form at 0x7fda60882950>]

In [73]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

['8321']


In [74]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [75]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

['anexos', 'language', 'numDOGC', 'seccion'] ['1', 'es_ES', '8321', '0']


'anexos=1&language=es_ES&numDOGC=8321&seccion=0'

## Recoge Nombre Secciones Sumario

In [76]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

'DISPOSICIONES'

## Recoge Nombre Secciones Sumario

In [77]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


   DISPOSICIONES 
   CARGOS Y PERSONAL
   ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
   ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
   ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA 
   ANUNCIOS VARIOS


In [78]:
df_secciones_sumarios

,Seccion
0,DISPOSICIONES
1,CARGOS Y PERSONAL
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA
5,ANUNCIOS VARIOS


## Recoge Valores para formar URLs Secciones Sumario

In [79]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=0
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=1
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=2
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=3
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=4
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=5


In [80]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [81]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [82]:
df_sumarios

,Seccion,URL_Seccion
0,DISPOSICIONES,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,CARGOS Y PERSONAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,ANUNCIOS VARIOS,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [83]:
df_sumarios['URL_Seccion'][4]

'https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8321&seccion=4'

## Recoge Items en Seccion Disposiciones

In [84]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

,Seccion,Item_Name,PDF_link
0,DISPOSICIONES,"RESOLUCIÓN TES/78/2021, de 15 de enero, por la...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,DISPOSICIONES,"RESOLUCIÓN TES/79/2021, de 15 de enero, por la...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,DISPOSICIONES,"RESOLUCIÓN TES/3526/2020, de 28 de diciembre, ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,DISPOSICIONES,"RESOLUCIÓN TES/3527/2020, de 28 de diciembre, ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,DISPOSICIONES,"EDICTO de 13 de enero de 2021, sobre acuerdos ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,DISPOSICIONES,"EDICTO de 13 de enero de 2021, sobre acuerdos ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,DISPOSICIONES,"EDICTO de 13 de enero de 2021, sobre un acuerd...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,DISPOSICIONES,"RESOLUCIÓN ARP/83/2021, de 13 de enero, por la...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [85]:
pdf_link

['https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891257&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891211&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891263&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891247&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891200&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891236&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891250&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891232&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=891233&type=01&

# DOUE

In [86]:
# --------------------------------------------------------------------------------------
# DOUE
# --------------------------------------------------------------------------------------

URL_HTML_resumen =  "https://eur-lex.europa.eu/oj/direct-access.html?locale=es"

## carga página HTML y genera árbol

response = requests.get(URL_HTML_resumen)
tree = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = tree.xpath('//*[@id="MainContent"]/div[2]/div[2]/div/table/tbody/tr[1]')
#print(secciones)



In [87]:
## Recoge Valores para formar URLs Secciones Sumario

df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion_name = seccion.xpath('./td[2]/a/text()')
    seccion_URL = seccion.xpath('./td[2]/a/@href')
    #seccion = re.sub('(\\r|\\n|\\t)+', '', seccion)
    #print(seccion_name)
    #print(seccion_URL)

for row in range(len(seccion_name)):
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion_name[row],
                                            'Seccion_link' : 'https://eur-lex.europa.eu' + seccion_URL[row][4:]},
                                            ignore_index=True)

In [88]:
df_secciones_sumarios

,Seccion,Seccion_link
0,L017,https://eur-lex.europa.eu/legal-content/ES/TXT...
1,L018,https://eur-lex.europa.eu/legal-content/ES/TXT...


In [89]:
[ print (row) for row in df_secciones_sumarios['Seccion_link']]

https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:017:TOC
https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:018:TOC


[None, None]

In [90]:
df_sumarios = pd.DataFrame()

for URL in df_secciones_sumarios['Seccion_link']:
    
    ### Recoge Items en Seccion Legislación

    #print(URL)
    response = requests.get(URL)
    sumario_HTML = html.fromstring(response.text)

    documentos = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/text()[1]')
    documentos_URL = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/@href')

    for row in range(len(documentos)):
        df_sumarios = df_sumarios.append({'Seccion': 'L' + documentos_URL[row][-7:-4],
                                        'Item_Name': documentos[row],
                                        'Item_Link': 'https://eur-lex.europa.eu' + documentos_URL[row][10:],
                                        'PDF_Link': ''},
                                        ignore_index=True)


In [91]:
df_sumarios

,Item_Link,Item_Name,PDF_Link,Seccion
0,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (UE) 2021/42 del Consejo, de 19 de en...",,L017
1,https://eur-lex.europa.eu/legal-content/ES/AUT...,Decisión N.,,L018


In [92]:
row = 0
for link in df_sumarios['Item_Link']:

    ### Recoge PDF link en Items Legislación

    #print(URL)
    response = requests.get(link)
    sumario_HTML = html.fromstring(response.text)

    PDF_Link = sumario_HTML.xpath('//*[@id="format_language_table_PDF_ES"]/@href')
    df_sumarios['PDF_Link'].iloc[row] = 'https://eur-lex.europa.eu' + PDF_Link[0][10:]

    row += 1
    print(PDF_link)

NameError: name 'PDF_link' is not defined

In [ ]:
df_sumarios = df_sumarios[['Seccion', 'Item_Name', 'Item_Link', 'PDF_Link']]
df_sumarios



## Recoge Valores para formar URLs Secciones Sumario

URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

## Concatena en df columnas Sccion y URL de sumarios 

df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '', item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

#df_disposiciones

def modifica_tabla_resultados_DOGC(tabla):
    tabla = tabla.reset_index(drop=True)
    tabla['Item_Name'] = '<a href=' + tabla['PDF_link'] + ' ' + 'target="_blank"' + '><div>' + tabla['Item_Name'] + '</div></a>' # Añade url link  pdf a Item_id
    tabla.drop(['PDF_link'], axis='columns', inplace=True)
    df_html = tabla.reset_index(drop=True).to_html(index='True', classes="table-hover", escape=False) # Utiliza Clase table-hover de Bootstrap
    df_html = df_html.replace("dataframe", "")  # Elimina clase por defecto dataframe
    df_html = df_html.replace('border="1"', 'border="2"')  # Incrementa tamaño línea borde tabla
    df_html = df_html.replace("<table", '<table style="font-size:12px; text-align: center; width: 100%" ') # Cambia tamaño fuente a 15px
    df_html = df_html.replace("<th>"+ tabla.columns[0], '<th style="text-align: center">'+ tabla.columns[0]) # Cambia alineación a header Columna 1
    df_html = df_html.replace("<th>"+ tabla.columns[1], '<th style="text-align: center">'+ tabla.columns[1]) # Cambia alineación a header Columna 2
    
    return df_html

st.header('')
st.header('DOGC disposiciones')


tabla_resultados_presentacion_DOGC = modifica_tabla_resultados_DOGC(df_disposiciones)

st.write(tabla_resultados_presentacion_DOGC, unsafe_allow_html=True)